# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109B Data Science 2: Advanced Topics in Data Science 

##  Final Project: Milestone 3 - Final Project [70 pts]


**Harvard University**<br/>
**Spring 2020**<br/>
**Group Members**: Fernando Medeiros, Mohammed Gufran Pathan, and Prerna Aggarwal<br/>

<hr style="height:2pt">

---

In [1]:
#RUN THIS CELL 
import requests
from IPython.core.display import HTML, display
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

<div class="theme"> Final Deliverables </div>

1. **Code Report:** You are expected to submit the code you developed as part of the course project. The commented code should be provided in report format. This means that each group in a Jupyter notebook should explain—in a clean and concise report fashion—how they proceeded at every step and coding/methodology  choices . The code report should have a structure that consists of an introduction, body and conclusion.
1. **Ignite Talk:** You will present the talk on 5/11, 5/12, or 5/13. Details to come for Ignite Talk guidelines.

[Final Project Guidelines](https://docs.google.com/document/d/1Zhmm9JP4FGQBi5abFiM22e5iXYo_rr7i_vbpW0-xt8A/edit)

In [2]:
# Baisc Python Libraries
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import timeit

plt.style.use("ggplot")

In [3]:
# Project Python Lybraries
from bs4 import BeautifulSoup
from collections import Counter
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from pandas.io.json import json_normalize
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import json
import lxml
import lzma
import nltk
import nltk.data
#nltk.download('punkt')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
from nltk import sent_tokenize
import tensorflow as tf
from tensorflow.keras import backend
from tensorflow.keras.layers import Input, SimpleRNN, Embedding, Dense, TimeDistributed, GRU,\
                                    Dropout, Bidirectional, Conv1D, BatchNormalization
from tensorflow.keras.models import Model, Sequential,model_from_json
from tensorflow.keras.utils import to_categorical

print(tf.keras.__version__)
print(tf.__version__)

2.2.4-tf
2.0.0


In [4]:
# Set seed for repeatable results
np.random.seed(123)
tf.random.set_seed(456)

<div class="theme"> Part 1: Data Prep </div>
Read `north_carolina.xz` into a pandas dataframe using the provided code below.

In [5]:
base_path = "./data/text"
state = 'north_carolina_text.xz'
f = lzma.open(os.path.join(base_path, state), "rb")
state_data = f.readlines()
f.close()

In [6]:
data_json = [json.loads(line) for line in state_data]
print(f'Flattening data for {state}')
data = json_normalize(data_json)

Flattening data for north_carolina_text.xz


In [7]:
for i in range(6): 
    data[f'casebody.data.opinions.type{i}'] = data['casebody.data.opinions'].map(lambda x: x[i]['type']\
                                                                                 if len(x)>=i+1 else None)
    data[f'casebody.data.opinions.text{i}'] = data['casebody.data.opinions'].map(lambda x: x[i]['text']\
                                                                                 if len(x)>=i+1 else None)

In [8]:
# fix date
data['decision_date_p'] = pd.to_datetime(data.decision_date, errors='coerce')
# add year of decision date to dataset
data['decision_year'] = data.decision_date_p.dt.year
# add month of decision date to dataset
data['decision_month'] = data.decision_date_p.dt.month

In [9]:
# select one year of cases
data_ym = data[(data.decision_year==1907) & (data.decision_month==11)]

In [10]:
# view top 5 cases
data_ym.head()

,id,url,name,name_abbreviation,decision_date,docket_number,first_page,last_page,citations,cites_to,...,casebody.data.opinions.text2,casebody.data.opinions.type3,casebody.data.opinions.text3,casebody.data.opinions.type4,casebody.data.opinions.text4,casebody.data.opinions.type5,casebody.data.opinions.text5,decision_date_p,decision_year,decision_month
1455,11270656,https://api.capapi.org/v1/cases/11270656/,W. L. PHILLIPS v. SALEM IRON WORKS,Phillips v. Salem Iron Works,1907-11-27,,209,218,"[{'type': 'official', 'cite': '146 N.C. 209'}]",[],...,None,None,None,None,None,None,None,1907-11-27,1907.0,11.0
1715,11253735,https://api.capapi.org/v1/cases/11253735/,STATE v. FRAZIER JONES,State v. Jones,1907-11-20,,466,472,"[{'type': 'official', 'cite': '145 N.C. 466'}]",[],...,None,None,None,None,None,None,None,1907-11-20,1907.0,11.0
1718,11253712,https://api.capapi.org/v1/cases/11253712/,STATE v. VERNON GODWIN,State v. Godwin,1907-11-06,,461,465,"[{'type': 'official', 'cite': '145 N.C. 461'}]",[],...,None,None,None,None,None,None,None,1907-11-06,1907.0,11.0
4399,11253399,https://api.capapi.org/v1/cases/11253399/,GREENLEAF-JOHNSON LUMBER COMPANY v. W. P. LEON...,Greenleaf-Johnson Lumber Co. v. Leonard,1907-11-06,,339,354,"[{'type': 'official', 'cite': '145 N.C. 339'}]",[],...,"CoNNOR, J.,\ndissenting: The exception to his ...",dissent,"Walker, J.,\ndissenting: I .agree fully in all...",None,None,None,None,1907-11-06,1907.0,11.0
4401,11253486,https://api.capapi.org/v1/cases/11253486/,T. J. McDUFFIE v. SEABOARD AIR LINE RAILWAY,McDuffie v. Seaboard Air Line Railway,1907-11-06,,397,399,"[{'type': 'official', 'cite': '145 N.C. 397'}]",[],...,None,None,None,None,None,None,None,1907-11-06,1907.0,11.0


In [11]:
# filter data and keep those cases with at least one opinion
# ----- CHANGE THE DATASET -----
data_f = data_ym[data_ym['casebody.data.opinions.type0'].notna()==True]

In [12]:
len(data_f)

44

In [13]:
print('1:', data_f[data_f['casebody.data.opinions.text0'].notna()].iloc[0][0])
print('2:', data_f[data_f['casebody.data.opinions.text1'].notna()].iloc[0][0])
print('3:', data_f[data_f['casebody.data.opinions.text2'].notna()].iloc[0][0])
print('4:', data_f[data_f['casebody.data.opinions.text3'].notna()].iloc[0][0])
print('5:', data_f[data_f['casebody.data.opinions.text4'].notna()].iloc[0][0])
print('6:', data_f[data_f['casebody.data.opinions.text5'].notna()].iloc[0][0])
# no cases with 7 opinion
#print('7:', data_f[data_f['casebody.data.opinions.text6'].notna()].iloc[0][0])

1: 11270656
2: 11253399
3: 11253399
4: 11253399
5: 11270334
6: 11270334


In [14]:
data_f['casebody.data.opinions.text0'].replace('\n', ' ', regex=True, inplace=True)
data_f['casebody.data.opinions.text1'].replace('\n', ' ', regex=True, inplace=True)
data_f['casebody.data.opinions.text2'].replace('\n', ' ', regex=True, inplace=True)
data_f['casebody.data.opinions.text3'].replace('\n', ' ', regex=True, inplace=True)
data_f['casebody.data.opinions.text4'].replace('\n', ' ', regex=True, inplace=True)
data_f['casebody.data.opinions.text5'].replace('\n', ' ', regex=True, inplace=True)

In [15]:
colNames = ['id','headnotes','opinions']
sentences = pd.DataFrame(columns = colNames)
for i, row in data_f.iterrows():
    tempDF = pd.DataFrame(columns = colNames)
    tempDF['id'] = [row.id]
    tempDF['headnotes'] = [sent_tokenize(str(row['casebody.data.head_matter']))]
    tempDF['opinions'] = [sent_tokenize(str(row['casebody.data.opinions.text0'])+\
                                        str(row['casebody.data.opinions.text1'])+\
                                        str(row['casebody.data.opinions.text2'])+\
                                        str(row['casebody.data.opinions.text3'])+\
                                        str(row['casebody.data.opinions.text4'])+\
                                        str(row['casebody.data.opinions.text5']))]
    sentences = sentences.append(tempDF, ignore_index=True)

sentences.to_json('./data/sentences.txt')

In [16]:
sentences = pd.read_json('./data/sentences.txt')

In [17]:
sentences

,id,headnotes,opinions
0,11270656,"[W. L. PHILLIPS v. SALEM IRON WORKS., (Filed 2...","[CoNNOR, J., after stating the facts: There ca..."
1,11253735,"[STATE v. FRAZIER JONES., (Filed 20 November, ...","[Walker,, J., after stating tbe case: T'be cha..."
2,11253712,"[STATE v. VERNON GODWIN., (Filed 6 November, 1...","[WalKee, L, after' stating the case: The charg..."
3,11253399,[GREENLEAF-JOHNSON LUMBER COMPANY v. W. P. LEO...,"[BbowN, J., We think there should be a new tri..."
4,11253486,"[T. J. McDUFFIE v. SEABOARD AIR LINE RAILWAY.,...","[BbowN, J., The provision made by the Corporat..."
5,11253812,"[STATE v. FRED CARMON., (Filed 27 November, 19...","[Walker, J., The jury might well have acquitte..."
6,11253451,[BLACKWELL’S DURHAM TOBACCO COMPANY v. THE AME...,"[CoNNOR, J., after stating the case: Plaintiff..."
7,11253755,"[STATE v. CEPHAS RAYNOR., (Filed 20 November, ...","[""Walker, J., The defendant was indicted and c..."
8,11253421,[G. H. BROCKENBROUGH v. MUTUAL RESERVE LIFE IN...,"[Claee, C. J., On 15 December, 1891, the defen..."
9,11253883,"[STATE v. MAJOR GUTHRIE., (Filed 27 November, ...","[BeowN, J. lYe find three assignments of error..."


In [18]:
sentences.headnotes[0]

['W. L. PHILLIPS v. SALEM IRON WORKS.',
 '(Filed 27 November, 1907).',
 '1.',
 'Employer and Employee — Safe Place to Work — Safe Appliances.',
 'The usual measure of duty imposed upon the employer requires him to furnish to his employee a reasonably safe place to work and such reasonably safe appliances as are known, approved and in general use.',
 '2.',
 'Same — Safety Appliances — Duty of Employer — Questions for Jury.',
 'When it is admitted or the jury find that standard safety appliances are known, approved and in general use in respect to the particular character of machinery furnished, or upon which plaintiff is employed, the law imposes the duty upon the employer to furnish such appliances, this being the standard of duty.',
 'When the evidence in this respect is conflicting, or the inference to be drawn from it doubtful, the question should be submitted to the jury, under proper instructions in regard to the standard of duty.',
 'Civil actioN, tried before Ferguson, J., and a

In [19]:
pd.DataFrame([sentences[0:1]][0])

,id,headnotes,opinions
0,11270656,"[W. L. PHILLIPS v. SALEM IRON WORKS., (Filed 2...","[CoNNOR, J., after stating the facts: There ca..."
